In [1]:
import re
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [2]:
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
path_raw = "../data/raw/cot"

## Articulo 21 COT ref: TRIBUNALES ORALES


In [3]:
with open(f'{path_raw}/Tribunales_orales.txt', 'r') as file:
    contenido_top = ''
    for line in file.readlines():
        contenido_top += line

In [4]:
regex_top=r"(?:(?P<Region>^[\w \']+)\:\n)|(?P<JG>^[\w. \-]+)\,\scon\s(?P<Jueces>[\w.\-]+)[a-z\-\s\,]+(?P<Competencia>\.|\s[\w. \-\,\']+)"
matches = re.findall(regex_top, contenido_top, re.MULTILINE)
#matches

# GENERA UNA ARREGLO DE LISTA DE 4 ELEMENTOS

# ELEMENTO 0 : REGIÓN
# ELEMENTO 1: TRIBUNAL ORAL DE ...
# ELEMENTO 2: Cantidad de Jueces
# ELEMENTO 3: Competencia (. es lo mismo de 3 0 7) o varias ciudades (separadas por , o y)


In [5]:
data_top=[]

for item in range(0,len(matches)):
    if matches[item][0] != '':
        region = matches[item][0].upper()
    else:    
        if matches[item][1] != '':
            ciudad = matches[item][1].upper() 
            if ciudad.find("TRIBUNAL") != -1:
                juzgado = ciudad
                
            else:
                juzgado = f"TRIBUNAL DE JUICIO ORAL EN LO PENAL {ciudad}"

        if matches[item][2] != '':
            cantidad_jueces = cleandata.transforma_numero(matches[item][2])
    
        if matches[item][3] == '.':
            competencia = ciudad
        
        else:  
            if matches[item][3] != '':
                competencia = matches[item][3].upper()
        
        competencia = competencia.replace(" Y ",",")
        competencia = competencia.replace(" E ",",")
        competencia = competencia.replace(".","")
        
        comunas = competencia.split(",")
        
        for comuna in comunas:
            data_top.append([region,juzgado,ciudad,cantidad_jueces,comuna.strip(),'ORAL'])
   
df_tribunal_oral = pd.DataFrame(data_top, columns = ['REGION','TRIBUNAL','ASIENTO','JUECES','COMUNA','TIPO JUZGADO'])


In [6]:
df_tribunal_oral['JUECES'] = df_tribunal_oral['JUECES'].fillna(0).astype(np.int8)

cols = df_tribunal_oral.select_dtypes(include = ["object"]).columns
df_tribunal_oral[cols] = df_tribunal_oral[cols].progress_apply(cleandata.elimina_tilde)

100%|██████████| 5/5 [00:00<00:00, 221.17it/s]


In [7]:
df_tribunal_oral

,REGION,TRIBUNAL,ASIENTO,JUECES,COMUNA,TIPO JUZGADO
0,PRIMERA REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,13,HUARA,ORAL
1,PRIMERA REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,13,CAMIÑA,ORAL
2,PRIMERA REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,13,COLCHANE,ORAL
3,PRIMERA REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,13,IQUIQUE,ORAL
4,PRIMERA REGION DE TARAPACA,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,IQUIQUE,13,POZO ALMONTE,ORAL
...,...,...,...,...,...,...
333,REGION DE ÑUBLE,TRIBUNAL DE JUICIO ORAL EN LO PENAL CHILLAN,CHILLAN,7,EL CARMEN,ORAL
334,REGION DE ÑUBLE,TRIBUNAL DE JUICIO ORAL EN LO PENAL CHILLAN,CHILLAN,7,PEMUCO,ORAL
335,REGION DE ÑUBLE,TRIBUNAL DE JUICIO ORAL EN LO PENAL CHILLAN,CHILLAN,7,YUNGAY,ORAL
336,REGION DE ÑUBLE,TRIBUNAL DE JUICIO ORAL EN LO PENAL CHILLAN,CHILLAN,7,TUCAPEL,ORAL


In [8]:
# Reset el index para realizar feather

df_tribunal_oral.reset_index(inplace = True)
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

# Guardamos dataset como archivo feather
df_tribunal_oral.to_feather(f'{path_interim}/TribunalOral_feather')
